In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
# !kaggle datasets list
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
! kaggle competitions download -c plant-seedlings-classification

  0% 0.00/5.13k [00:00<?, ?B/s]
100% 5.13k/5.13k [00:00<00:00, 2.17MB/s]
 91% 78.0M/86.0M [00:00<00:00, 72.8MB/s]
100% 86.0M/86.0M [00:01<00:00, 88.6MB/s]
 99% 1.59G/1.60G [00:21<00:00, 81.1MB/s]
100% 1.60G/1.60G [00:21<00:00, 80.0MB/s]


In [0]:
from zipfile import ZipFile 
  
# file names
file_names = ["train.zip","test.zip"]
  
# extraction
for file in file_names:
    with ZipFile(file, 'r') as zip:  
        # extracting all the files 
        print('Extracting all the {0} files now...'.format(file)) 
        zip.extractall() 
        print('Done!') 

Extracting all the train.zip files now...
Done!
Extracting all the test.zip files now...
Done!


Importing all the necessary libraries

In [0]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import numpy as np
import random
import os
import sys
import cv2
from keras.utils import to_categorical
import matplotlib

Using TensorFlow backend.


In [0]:
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()

def readTrainData(trainDir):
    data = []
    labels = []
    # loop over the input images
    dirs = os.listdir(trainDir) 
    # print(dirs)
    i=0
    for dir in dirs:
        absDirPath = os.path.join(trainDir, dir)
        images = os.listdir(absDirPath)
        Classes=le.fit_transform(dirs)
        Cls=Classes[i]
        i=i+1
        for imageFileName in images:
            # load the image, pre-process it, and store it in the data list
            imageFullPath = os.path.join(trainDir, dir, imageFileName)
            # print(imageFullPath)
            img = load_img(imageFullPath)
            arr = img_to_array(img)  # Numpy array with shape (233,233,3)
            arr = cv2.resize(arr, (128,128)) #Numpy array with shape (HEIGHT, WIDTH,3)
            #print(arr.shape) 
            data.append(arr)
            label = Cls
            labels.append(label)
    return data, labels



In [0]:
BASE_DATA_FOLDER = "."
TRAin_DATA_FOLDER = os.path.join(BASE_DATA_FOLDER, "train")
TRAin_DATA_FOLDER

'./train'

Building the model

In [0]:
NUM_CLASSES = 12
WIDTH = 128
HEIGHT = 128
DEPTH = 3
inputShape = (WIDTH, HEIGHT, DEPTH)
# initialize number of epochs to train for, initial learning rate and batch size
EPOCHS = 15
INIT_LR = 1e-3
BS = 32
    
model = Sequential()
    # The CONV  layer will learn 20 convolution filters, each of which are 5×5.
model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
    # We then apply a ReLU activation function followed by 2×2 max-pooling in both 
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.001))
    # second set of CONV => RELU => POOL layers and a Dropout layer
model.add(Conv2D(50, (5, 5), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.001))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

model.add(Dense(output_dim=12))
model.add(Activation("softmax"))

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

W0825 01:51:26.740725 140632606508928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0825 01:51:26.793892 140632606508928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 01:51:26.805896 140632606508928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0825 01:51:26.843087 140632606508928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0825 01:51:26.846855 140632606508928 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 20)      1520      
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 20)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 20)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 50)        25050     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 50)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 64, 50)        0         
__________

In [0]:
X, Y = readTrainData(TRAin_DATA_FOLDER)
# Normalisation
X = np.array(X, dtype="float") / 255.0
Y = np.array(Y)
# convert the labels from integers to vectors
Y =  to_categorical(Y, num_classes=12)

In [0]:
(trainX, valX, trainY, valY) = train_test_split(X,Y,test_size=0.25, random_state=10)

In [0]:
model.fit(trainX, trainY, batch_size=BS,validation_data=(valX, valY), epochs=EPOCHS, verbose=1)

W0825 01:54:50.503249 140632606508928 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3562 samples, validate on 1188 samples
Epoch 1/15
3562/3562 [==============================] - 17s 5ms/step - loss: 2.0625 - acc: 0.3175 - val_loss: 1.5907 - val_acc: 0.4209
Epoch 2/15
3562/3562 [==============================] - 10s 3ms/step - loss: 1.2508 - acc: 0.5654 - val_loss: 1.2674 - val_acc: 0.5556
Epoch 3/15
3562/3562 [==============================] - 10s 3ms/step - loss: 0.9217 - acc: 0.6791 - val_loss: 1.0976 - val_acc: 0.6305
Epoch 4/15
3562/3562 [==============================] - 10s 3ms/step - loss: 0.7556 - acc: 0.7389 - val_loss: 0.9458 - val_acc: 0.6869
Epoch 5/15
3562/3562 [==============================] - 10s 3ms/step - loss: 0.5880 - acc: 0.7981 - val_loss: 0.9206 - val_acc: 0.6995
Epoch 6/15
3562/3562 [==============================] - 10s 3ms/step - loss: 0.4182 - acc: 0.8608 - val_loss: 0.9677 - val_acc: 0.6919
Epoch 7/15
3562/3562 [==============================] - 10s 3ms/step - loss: 0.3396 - acc: 0.8897 - val_loss: 0.9203 - val_acc: 0.7281
Epoch 8

In [0]:
score=model.evaluate(valX, valY) 
print('Test Accuracy:',score[1])

1188/1188 [==============================] - 1s 825us/step
Test Accuracy: 0.7356902356902357


In [0]:
# Trying Image augumentation for improving the accuracy
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

model.fit_generator(aug.flow(trainX, trainY,batch_size=BS),validation_data=(valX, valY),steps_per_epoch=len(trainX) // BS, epochs=EPOCHS, verbose=1)

Epoch 1/15
111/111 [==============================] - 15s 132ms/step - loss: 1.4065 - acc: 0.5526 - val_loss: 0.8902 - val_acc: 0.7163
Epoch 2/15
111/111 [==============================] - 14s 125ms/step - loss: 1.0256 - acc: 0.6451 - val_loss: 0.8116 - val_acc: 0.7407
Epoch 3/15
111/111 [==============================] - 14s 125ms/step - loss: 0.9518 - acc: 0.6676 - val_loss: 0.6758 - val_acc: 0.7769
Epoch 4/15
111/111 [==============================] - 14s 124ms/step - loss: 0.8503 - acc: 0.7144 - val_loss: 0.6745 - val_acc: 0.7887
Epoch 5/15
111/111 [==============================] - 14s 126ms/step - loss: 0.8205 - acc: 0.7226 - val_loss: 0.7072 - val_acc: 0.7668
Epoch 6/15
111/111 [==============================] - 14s 124ms/step - loss: 0.7576 - acc: 0.7428 - val_loss: 0.6495 - val_acc: 0.7811
Epoch 7/15
111/111 [==============================] - 14s 126ms/step - loss: 0.7339 - acc: 0.7472 - val_loss: 0.5593 - val_acc: 0.8173
Epoch 8/15
111/111 [==============================] - 1

In [0]:
from keras.utils import np_utils

y_pred_class=model.predict_classes(valX)
y_pred=np_utils.to_categorical(y_pred_class,12)

from sklearn.metrics import classification_report

Score=model.evaluate(valX, valY)
print('Test Accuracy:',Score[1],'\n')
print(classification_report(valY, y_pred))

1188/1188 [==============================] - 1s 772us/step
Test Accuracy: 0.8291245791245792 

              precision    recall  f1-score   support

           0       0.39      0.28      0.33        64
           1       0.86      0.93      0.89        95
           2       0.89      0.93      0.91        85
           3       0.92      0.91      0.91       165
           4       0.72      0.88      0.79        56
           5       0.92      0.84      0.88       122
           6       0.76      0.76      0.76       156
           7       0.86      0.98      0.92        51
           8       0.76      0.92      0.83       126
           9       0.78      0.63      0.70        68
          10       0.94      0.87      0.90       106
          11       0.89      0.84      0.86        94

   micro avg       0.83      0.83      0.83      1188
   macro avg       0.81      0.81      0.81      1188
weighted avg       0.83      0.83      0.82      1188
 samples avg       0.83      0.83      

In [0]:
dirs = os.listdir(TRAin_DATA_FOLDER) 
Classes=le.fit_transform(dirs)
for i in range(len(dirs)):
  print(dirs[i],':',Classes[i])

Charlock : 1
Black-grass : 0
Loose Silky-bent : 6
Shepherds Purse : 9
Common Chickweed : 3
Sugar beet : 11
Small-flowered Cranesbill : 10
Common wheat : 4
Fat Hen : 5
Maize : 7
Scentless Mayweed : 8
Cleavers : 2


In [0]:
# Final Train
model.fit_generator(aug.flow(X, Y,batch_size=BS),validation_data=(valX, valY),steps_per_epoch=len(X) // BS, epochs=EPOCHS, verbose=1)
y_pred_class=model.predict_classes(X)
y_pred=np_utils.to_categorical(y_pred_class,12)
Score=model.evaluate(X, Y)
print('Test Accuracy:',Score[1],'\n')
print(classification_report(Y, y_pred))

Epoch 1/15
148/148 [==============================] - 19s 130ms/step - loss: 0.5447 - acc: 0.8136 - val_loss: 0.4628 - val_acc: 0.8384
Epoch 2/15
148/148 [==============================] - 18s 124ms/step - loss: 0.5308 - acc: 0.8177 - val_loss: 0.4115 - val_acc: 0.8577
Epoch 3/15
148/148 [==============================] - 18s 124ms/step - loss: 0.5246 - acc: 0.8170 - val_loss: 0.3678 - val_acc: 0.8662
Epoch 4/15
148/148 [==============================] - 18s 124ms/step - loss: 0.4686 - acc: 0.8351 - val_loss: 0.3718 - val_acc: 0.8746
Epoch 5/15
148/148 [==============================] - 18s 123ms/step - loss: 0.4790 - acc: 0.8321 - val_loss: 0.3477 - val_acc: 0.8805
Epoch 6/15
148/148 [==============================] - 19s 126ms/step - loss: 0.4650 - acc: 0.8392 - val_loss: 0.3470 - val_acc: 0.8838
Epoch 7/15
148/148 [==============================] - 19s 126ms/step - loss: 0.4464 - acc: 0.8392 - val_loss: 0.3014 - val_acc: 0.8880
Epoch 8/15
148/148 [==============================] - 1